In [3]:
import kratos
from agno.agent import Agent
from agno.models.lmstudio import LMStudio
from agno.tools.youtube import YouTubeTools
from agno.tools.yfinance import YFinanceTools

# Creating a few agents using the Agno framework for building agents
# All the llm models here use LMStudio as the provider and connect to local API server
# I'm using qwen/qwen3-4b model for both agents, but you can use any other model that you have access to.

finance_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosFinance",
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="You are Kratos Finance Agent, specialized in stock market analysis and financial data. Use YFinance to get current stock prices, analyst recommendations, and company information. When users ask about stocks, always provide current prices and relevant financial metrics. Be precise with numbers and explain what the data means for investment decisions.",
)


youtube_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosYouTube",
    tools=[YouTubeTools()],
    instructions="You are Kratos YouTube Agent, specialized in analyzing YouTube videos and transcripts. Use YouTube tools to extract video information, transcripts, and analyze content. When users mention YouTube videos, provide detailed analysis of the content, key points, and insights from the transcript. Be thorough in your analysis and highlight important information.",
)


In [2]:
# Submit these agents so that they are ready to be used at a later point - none of
# these models are actually loaded in memory right now and thus no resources are wasted.

print(kratos.submit(finance_agent, name="agent-finance", dependencies=["yfinance"]))
print(kratos.submit(youtube_agent, name="agent-youtube", dependencies=["youtube-transcript-api"]))

agent-finance
agent-youtube


In [3]:
# The name parameter we used in submit() acts as an identifier as to which agent do you want to invoke.

kratos.invoke(name="agent-finance", task="How has the trend been for NVIDIA stock?")

<think>
Okay, the user is asking about the trend for NVIDIA stock. To answer this, I need to figure out what data points are relevant. First, the current stock price would be a good starting point. So I should call the get_current_stock_price function with the symbol NVDA.
But wait, the user is asking about the trend, not just the current price. Maybe they want to know if the stock has been going up or down recently. However, the available functions don't include historical price data. The tools provided are for current price, company info, and analyst recommendations. 
So I can't get historical trends directly. But perhaps I can check the current price and maybe analyst recommendations to give some context. Let me start by getting the current price. Then, if there's analyst sentiment, that could influence the trend. Also, company info might have recent news or developments affecting the stock.
Wait, the user might be expecting a summary of recent performance. Since I can't retrieve hi

In [6]:
kratos.invoke(name="agent-youtube", task="Summarize the transcript of this video: https://www.youtube.com/watch?v=dQw4w9WgXcQ")

<think>
Okay, the user wants me to summarize the transcript of the YouTube video with the URL https://www.youtube.com/watch?v=dQw4w9WgXcQ. First, I need to get the transcript from that video. Looking at the tools provided, there's a function called get_youtube_video_captions which requires the video URL. So I should call that function first to retrieve the captions.
Once I have the transcript, I can proceed to summarize it. But wait, the user might also want timestamps if needed, but the main request is a summary. However, I should check if the transcript is available. The video URL provided is for the Rickroll classic, which is well-known. I remember that this video is a famous joke, so the transcript might be straightforward.
But to follow the steps properly, I need to first get the captions using the get_youtube_video_captions function. Then, once I have the transcript text, I can analyze it and provide a summary. The user didn't mention anything about timestamps, so maybe the summa

In [4]:
from concurrent.futures import ThreadPoolExecutor, wait

print(f"\n🚀 Starting 3 concurrent invocations where each instance of an agent will have it's own model loaded in memory")

tasks = [
    "Stock price of NVIDIA",
    "Stock price of Apple",
    "Stock price of Tesla",
]

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(kratos.invoke, 'agent-finance', task, False) for task in tasks]

    print(f"\n🔄 Waiting for all agents to complete...")
    wait(futures)

    print(f"\n🔄 All agents have completed their tasks")

print("\n\nQ&A:")


🚀 Starting 3 concurrent invocations where each instance of an agent will have it's own model loaded in memory

🔄 Waiting for all agents to complete...

🔄 All agents have completed their tasks


Q&A:
